exploratoryanalysis.R
Jay Sayre - sayrejay (at) gmail,

NOTE: cleanuptextable.py MUST BE RUN AFTER RUNNING THIS SCRIPT TO UPDATE TABLES IN PAPER

INPUTS: 
"municipality_level_DATASET.csv" - As titled, produced by dataassembly.py

"mun_level_isic4dig_DATASET.csv" - Same as above, except uses ISIC 4 digit and only empresas data (not IPUMS), produced by dataassembly.py

"municipality_occupation_level_DATASET.csv" - As titled, produced by dataassembly.py

OUTPUTS:

"../Plots/munregs.tex" - Municipality level results

"../Plots/mun4digregs.tex" - Municipality level results (ISIC 4 digit/no IPUMS occupation)

"../Plots/munoccregs.tex" - Occupation level results

In [1]:
library(ggplot2)
library(texreg)
#install.packages("texreg", repos='http://cran.us.r-project.org')
library(tikzDevice)
#install.packages("tikzDevice", repos='http://cran.us.r-project.org')

## Directory information
setwd("~/Dropbox/College/DR_Paper/cafta-dr/Output/")
#setwd("D:/Dropbox/Dropbox (Personal)/College/DR_Paper/")
plotdir <- "../Plots/"

## INPUTS 
mundf <- read.csv("municipality_level_DATASET.csv")
munisic4df <- read.csv("mun_level_isic4dig_DATASET.csv")
munoccdf <- read.csv("municipality_occupation_level_DATASET.csv")

## OUTPUTS 
munlevelresults <- paste(plotdir,"munregs.tex",sep="")
mun4diglevelresults <- paste(plotdir,"mun4digregs.tex",sep="")
occupationlevelresults <- paste(plotdir,"munoccregs.tex",sep="")

Version:  1.36
Date:     2015-12-08
Author:   Philip Leifeld (Eawag & University of Bern)

Please cite the JSS article in your publications -- see citation("texreg").


In [2]:
### Cleanup variables in municipality level dataset
mundf$chnginc1 <- log(1+mundf$incforocc13)-log(1+mundf$inc02)
mundf$chnginc2 <- log(1+mundf$totalhhincpercapita13)-log(1+mundf$inc02)
mundf$chnginc3 <- log(1+mundf$firstsourceofincome13)-log(1+mundf$inc02)
mundf$chnginc4 <- log(1+mundf$incpercapita13)-log(1+mundf$inc02)
mundf$prov <- factor(mundf$prov)
mundf$chngtrf <- log(1+mundf$duty13)-log(1+mundf$duty02)

In [3]:
### Cleanup variables in municipality level (ISIC 4) dataset
munisic4df$chnginc1 <- log(1+munisic4df$incforocc13)-log(1+munisic4df$inc02)
munisic4df$chnginc2 <- log(1+munisic4df$totalhhincpercapita13)-log(1+munisic4df$inc02)
munisic4df$chnginc3 <- log(1+munisic4df$firstsourceofincome13)-log(1+munisic4df$inc02)
munisic4df$chnginc4 <- log(1+munisic4df$incpercapita13)-log(1+munisic4df$inc02)
munisic4df$prov <- factor(munisic4df$prov)
munisic4df$chngtrf <- log(1+munisic4df$duty13)-log(1+munisic4df$duty02)

In [4]:
### Cleanup variables in municipality/occupation level dataset
#munoccdf$mean=rowMeans(munoccdf[,c("incforocc13", "firstsourceofincome13",
#                                  "totalhhincpercapita13","incpercapita13")],
#                       na.rm=TRUE)
#munoccdf$chnginc5 <- munoccdf$mean-munoccdf$inc2002

munoccdf$chnginc1 <- munoccdf$incforocc13-munoccdf$inc2002
munoccdf$chnginc2 <- munoccdf$firstsourceofincome13-munoccdf$inc2002
munoccdf$chnginc3 <- munoccdf$totalhhincpercapita13-munoccdf$inc2002
munoccdf$chnginc4 <- munoccdf$incpercapita13-munoccdf$inc2002
munoccdf$chngtrf <- munoccdf$duty13-munoccdf$duty02
munoccdf$lgchng <- log(-munoccdf$chngtrf+1)
munoccdf$lgdty02 <-log(munoccdf$duty02+1)
munoccdf$chngwrk <- munoccdf$numworkers10-munoccdf$numworkers02
munoccdf$occ <- factor(munoccdf$occ)
munoccdf$munocc <- factor(munoccdf$munocc)

In [5]:
### Municipality level regressions
#colnames(mundf)
munreg1 <- lm(chnginc1~chngtrf+prov, data = mundf)
munreg2 <- lm(chnginc3~chngtrf, data = mundf)
munreg3 <- lm(chnginc3~chngtrf+prov, data = mundf)

#summary(munreg1)
#summary(munreg2)
#summary(munreg3)

#plot(mundf$chnginc3,mundf$chngtrf)

texreg(list(munreg1,munreg2,munreg3),
        file=munlevelresults,
        stars = c(0.01, 0.05,0.10),
        caption="Municipality level regressions",
        caption.above = TRUE, 
        omit.coef="prov",
        custom.model.names=c("(1)","(2)","(3)"))

The table was written to the file '../Plots/munregs.tex'.



In [6]:
### Municipality level (ISIC 4 digit) regressions
#colnames(mundf)
mun4reg1 <- lm(chnginc1~chngtrf+prov, data = munisic4df)
mun4reg2 <- lm(chnginc3~chngtrf, data = munisic4df)
mun4reg3 <- lm(chnginc3~chngtrf+prov, data = munisic4df)

#summary(mun4reg1)
#summary(mun4reg2)
#summary(mun4reg3)

texreg(list(mun4reg1,mun4reg2,mun4reg3),
        file=mun4diglevelresults,
        stars = c(0.01, 0.05,0.10),
        caption="Municipality level regressions",
        caption.above = TRUE, 
        omit.coef="prov",
        custom.model.names=c("(1)","(2)","(3)"))

The table was written to the file '../Plots/mun4digregs.tex'.



In [7]:
### Municipality/occupation level regressions
#colnames(munoccdf)
reg1 <- lm(chnginc2~chngtrf,data = munoccdf)
reg2 <- lm(chnginc2~chngtrf+munocc,data = munoccdf)
reg3 <- lm(chnginc2~chngtrf+chngwrk,data = munoccdf)
reg4 <- lm(chnginc2~chngtrf+chngwrk+munocc,data = munoccdf)

#summary(reg1)
#summary(reg2)
#summary(reg3)
#summary(reg4)
#plot(munoccdf$chnginc2,munoccdf$chngtrf)

texreg(list(reg1,reg2,reg3,reg4),
        file=occupationlevelresults,
        stars = c(0.01, 0.05,0.10),
        caption="Municipality/occupation level regressions",
        caption.above = TRUE, 
        omit.coef="munocc|z",
        custom.model.names=c("(1)","(2)","(3)","(4)"))

The table was written to the file '../Plots/munoccregs.tex'.

